In [ ]:
import fiona 
import pandas as pd 
import geopandas as gpd 
import random
import os
import folium
import io
from PIL import Image

In [ ]:
path = os.path.dirname(os.path.abspath(os.path.dirname(__name__)))

fiona.drvsupport.supported_drivers['libkml'] = 'rw' 
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

path = path + "/arquivos/"
kmz_file_path = path + "linhas_observatório.kmz"
kml_file_path = path + "linhas_tratadas.kml"

layers = gpd.list_layers(kml_file_path)

gdf_list = []
for layer in fiona.listlayers(kml_file_path):
    if layer == 'Linhas prontas':
        continue
    gdf = gpd.read_file(kml_file_path, driver='LIBKML', layer=layer)
    gdf_list.append(gdf)

gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

gdf_filtered = gdf.query("Name != ''")
gdf_filtered = gdf_filtered.query("Name != '1'")
gdf = gdf_filtered.query("Name != 'inativo'")
gdf

In [ ]:
m = folium.Map([-16.737, -43.8647])

shapefile_moc = path + "bairros_sirgas.shp"

gdf_moc = gpd.read_file(shapefile_moc)

gdf_moc

In [ ]:
print(path)
shapefile_ibge = path +"IBGE/" + "MG_Municipios_2022.shp"
gdf_ibge = gpd.read_file(shapefile_ibge)

gdf_ibge

In [ ]:
map_center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]
m = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB Voyager')
folium.Rectangle(
    bounds=[[-90, -180], [90, 180]],
    color="#F9FFBC",
    fill=True,
    fill_color="#F9FFBC",
    fill_opacity=1
).add_to(m)
folium.TileLayer().add_to(m)

folium.GeoJson(
    gdf_moc,
    style_function=lambda feature: {
        'fillColor': 'white',  # Cor de preenchimento do polígono
        'color': 'black',         # Cor das bordas
        'weight': 0.7,           # Espessura das bordas
        'fillOpacity': 0.5,      # Opacidade do preenchimento
    },
    name = 'Bairros'
).add_to(m)

m

In [ ]:
def randon_color():
    cor = "#{:06x}".format(random.randint(0, 0xFFFFFF))
    return cor

In [ ]:
def add_line_to_map(linha, geometry, group):
    cor = randon_color()
    folium.PolyLine(
        locations=[(lat, lon) for lon, lat in zip(geometry.xy[0], geometry.xy[1])],
        color= cor,
        weight=2.5,
        opacity=1,
        tooltip = linha
    ).add_to(group)

def add_line_to_map_sem_grupo(linha, geometry, cor):
    folium.PolyLine(
        locations=[(lat, lon) for lon, lat in zip(geometry.xy[0], geometry.xy[1])],
        color= cor,
        weight=3,
        opacity=1,
        tooltip = linha
    ).add_to(m)

In [ ]:
map_center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]
m = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB Voyager')
folium.GeoJson(
    gdf_moc,
    style_function=lambda feature: {
        'fillColor': 'white',  # Cor de preenchimento do polígono
        'color': 'black',         # Cor das bordas
        'weight': 0.7,           # Espessura das bordas
        'fillOpacity': 0.5,      # Opacidade do preenchimento
    },
    name = 'Bairros'
).add_to(m)

legenda_html = """
<div style="
     position: fixed; 
     top : 50px; right: 80px; width: 400px; height: auto; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <div>
        <h4 style="text-align: center; font-weight: bold;">LINHAS DE ÔNIBUS DE MONTES CLAROS - MG <br>(PADRÃO BOM)</h4>
     </div>
</div>
"""

legenda_html += """
     <div style="
     position: fixed; 
     top : 250px; right: 150px; width: 250px; height: auto; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <b>Legenda</b><br>
     <div style="display: flex; justify-content: space-between;">
     <div style="flex: 1;">
"""

linhas_bom = ['1701', '1702', '2202', '2601', '2603', '4601', '4603', '5801', '5802', '7103']
cores = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'black', '#37FFEE', 'pink']
count = 0
for index, row in gdf.iterrows():
    separacao = row.Name.strip().split('-')
    if len(separacao) != 2:
        continue
    linha, sentido = separacao
    if sentido.strip() == 'IDA' and linha.strip() in linhas_bom:
        cor = randon_color()
        add_line_to_map_sem_grupo(linha, row.geometry, cores[count])
        if count == len(linhas_bom)/2:
            legenda_html += '</div><div style="flex: 1;">'
        legenda_html += f"""
            <div style="display: flex; align-items: center; width: 120px; margin-bottom: 5px;">
                <div style="width: 40px; height: 10px; background-color: {cores[count]}; margin-right: 5px;"></div>
                {linha}
            </div>
        """
        count+=1

legenda_html += "</div></div></div>"

In [ ]:
m.get_root().html.add_child(folium.Element(legenda_html))
m.save("mapa_com_legenda_linhas3.html")

import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('linhas_bom_2.png')

In [ ]:
map_center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]
m = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB Voyager')

folium.GeoJson(
    gdf_moc,
    style_function=lambda feature: {
        'fillColor': 'white',  # Cor de preenchimento do polígono
        'color': 'black',         # Cor das bordas
        'weight': 0.7,           # Espessura das bordas
        'fillOpacity': 0.5,      # Opacidade do preenchimento
    },
    name = 'Bairros'
).add_to(m)

legenda_html = """
<div style="
     position: fixed; 
     top : 50px; right: 80px; width: 400px; height: auto; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <div>
        <h4 style="text-align: center; font-weight: bold;">LINHAS DE ÔNIBUS DE MONTES CLAROS - MG <br>(PADRÃO INSUFICIENTE)</h4>
     </div>
</div>
"""

legenda_html += """
     <div style="
     position: fixed; 
     top : 250px; right: 150px; width: 250px; height: auto; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <b>Legenda</b><br>
     <div style="display: flex; justify-content: space-between;">
     <div style="flex: 1;">
"""

linhas_insuficiente = ['1501', '1601', '2201', '2203', '2604', '3301', '4701', '5101', '5601', '5702', '5803', '5901', '5902', '6201R', '6202', '6404', '6604', '6901', '7101', '8201']
cores = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'black', '#37FFEE', 'pink', 'red', 'blue', 'green', 'purple', 'orange', 'brown', 'black', '#37FFEE', 'pink', 'red', 'blue', 'green', 'purple', 'orange', 'brown', 'black', '#37FFEE', 'pink']
count = 0
for index, row in gdf.iterrows():
    separacao = row.Name.strip().split('-')
    if len(separacao) != 2:
        continue
    linha, sentido = separacao
    if sentido.strip() == 'IDA' and linha.strip() in linhas_insuficiente:
        cor = randon_color()
        print(linha)
        add_line_to_map_sem_grupo(linha, row.geometry, cores[count])
        if count == len(linhas_insuficiente)/2:
            legenda_html += '</div><div style="flex: 1;">'
        legenda_html += f"""
            <div style="display: flex; align-items: center; width: 120px; margin-bottom: 5px;">
                <div style="width: 40px; height: 10px; background-color: {cor}; margin-right: 5px;"></div>
                {linha}
            </div>
        """
        count+=1
legenda_html += "</div></div></div>"

In [ ]:

m.get_root().html.add_child(folium.Element(legenda_html))

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('linhas_insuficiente_2.png')